In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="project1hp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "MLCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_modes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--max_iter": uniform(10,30),
        "--C": choice(0.5, 1, 1.5)
    }
)   

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,  # The slack factor to allow for some variability
    evaluation_interval=1,  # How often to evaluate the performance
    delay_evaluation=5  # Number of iterations to wait before evaluating
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',  
                  script='train.py',  # Your training script
                  compute_target= 'Proj1ML',
                  Environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config = src,  # Your script run configuration
    hyperparameter_sampling= ps,
    policy=policy,  # Include the early stopping policy
    primary_metric_name='accuracy',  # Metric to optimize
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  # Goal to optimize
    max_total_runs=20,  # Total number of runs
    max_concurrent_runs=4, # Number of concurrent runs
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

In [ ]:
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
print(best_run.get_metrics())

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [ ]:
from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)


In [ ]:
# Submit your automl run
experiment = Experiment(ws, 'project1automl')
run = experiment.submit(config=automl_config, show_out=True)

In [ ]:
# Retrieve and save your best automl model.

best_auto_run, model = run.get_output()
best_auto_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_auto_run)

In [ ]:
print(best_auto_run.get_details())


In [ ]:
print(best_auto_run.get_metrics())

In [ ]:
#cluster.delete()